# Evaluating truth vs sourmash gather



In [55]:
import csv
import sys
sys.path.insert(0, '../2018-ncbi-lineages/')
import ncbi_taxdump_utils
import os

In [23]:
taxfoo = ncbi_taxdump_utils.NCBI_TaxonomyFoo()

taxfoo.load_nodes_dmp('../2018-ncbi-lineages/genbank/nodes.dmp.gz')
taxfoo.load_names_dmp('../2018-ncbi-lineages/genbank/names.dmp.gz')

want_taxonomy = ['superkingdom', 'phylum', 'order', 'class', 'family', 'genus', 'species']

In [24]:
from sourmash_lib.lca import lca_utils, command_index

def format_lineage(lineage_tup):
    return ";".join(lca_utils.zip_lineage(lineage_tup))

## Sourmash gather: mapping from accession -> lineages

This is important for dealing with the CSV output of `sourmash gather`; `sourmash lca gather`
comes with the taxonomy already in the CSV output.

In [25]:
acc_to_lineage, num_rows = command_index.load_taxonomy_assignments('gather-lineages.csv', start_column=3)
print('loaded {} rows'.format(num_rows))

loaded 798 rows


examining spreadsheet headers...
** assuming column 'accession' is identifiers in spreadsheet


In [26]:
def get_lineage_by_acc(acc):
    acc = acc.split(' ')[0].split('.')[0]
    return acc_to_lineage.get(acc, None)

# example for 'get_lineage_by_acc'
#lineage = get_lineage_by_acc('NC_000917')
#print(lineage)
#print(format_lineage(lineage))

In [27]:
def load_sourmash_csv(filename):
    with open(filename, 'rt') as fp:
        r = csv.DictReader(fp)
        rows = list(r)
    return rows

#load_sourmash_csv('output/Huttenhower_HC1.fasta.gz.scaled10k.k51.gather.matches.csv')[:1]

In [28]:
def make_gather_lineages(filename):
    rows = load_sourmash_csv(filename)
    rows2 = []
    for d in rows:
        name = d['name']
        lineage = get_lineage_by_acc(name)
        if lineage is None:
            print('ZZZ found no lineage for {}'.format(name))
        d2 = dict(d)
        d2['lineage'] = lineage
        rows2.append(d2)
        
    return rows2

make_gather_lineages('output/Huttenhower_HC1.fasta.gz.scaled10k.k51.gather.matches.csv')[:1]

[{'f_match': '0.03466386554621849',
  'f_orig_query': '0.008465879938429965',
  'f_unique_to_query': '0.008465879938429965',
  'filename': 'inputs/databases/genbank-k51.sbt.json',
  'intersect_bp': '330000',
  'lineage': (LineagePair(rank='superkingdom', name='Bacteria'),
   LineagePair(rank='phylum', name='Bacteroidetes'),
   LineagePair(rank='class', name='Chitinophagia'),
   LineagePair(rank='order', name='Chitinophagales'),
   LineagePair(rank='family', name='Chitinophagaceae'),
   LineagePair(rank='genus', name='Chitinophaga'),
   LineagePair(rank='species', name='Chitinophaga pinensis')),
  'md5': 'd06a405d39c8ebed579540db2994afe8',
  'name': 'NC_013132.1 Chitinophaga pinensis DSM 2588, complete genome'}]

## Truth files: loading in & fleshing out NCBI taxonomy

Also needed :).

In [29]:
def load_truth_file(filename):
    with open(filename, 'rt') as fp:
        lines = fp.readlines()
        
    lines = [ x.strip() for x in lines ]
    lines = [ x.split('\t') for x in lines ]
    
    rows = []
    for x in lines:
        taxid, a, b, rank, name = x
        taxid = int(taxid)
        rows.append((taxid, a, b, rank, name))
    return rows
    
print(load_truth_file('truth_sets/species/Huttenhower_HC1_TRUTH.txt')[:5])

[(84980, '1.00000', '0.01000', 'species', 'Desulfotalea psychrophila'), (1428, '1.00000', '0.01000', 'species', 'Bacillus thuringiensis'), (1496, '1.00000', '0.01000', 'species', 'Peptoclostridium difficile'), (1525, '1.00000', '0.01000', 'species', 'Moorella thermoacetica'), (1423, '1.00000', '0.01000', 'species', 'Bacillus subtilis')]


In [38]:
def make_lineage_from_taxid(taxid):
    lineage_d = taxfoo.get_lineage_as_dict(taxid, want_taxonomy)
    
    lineage = []
    for rank in lca_utils.taxlist():
        name = lineage_d.get(rank, 'unassigned')
        lineage_pair = lca_utils.LineagePair(rank, name)
        lineage.append(lineage_pair)
    return tuple(lineage)

make_lineage_from_taxid(84980)

(LineagePair(rank='superkingdom', name='Bacteria'),
 LineagePair(rank='phylum', name='Proteobacteria'),
 LineagePair(rank='class', name='Deltaproteobacteria'),
 LineagePair(rank='order', name='Desulfobacterales'),
 LineagePair(rank='family', name='Desulfobulbaceae'),
 LineagePair(rank='genus', name='Desulfotalea'),
 LineagePair(rank='species', name='Desulfotalea psychrophila'))

In [39]:
def make_truth_lineages(filename):
    rows = load_truth_file(filename)
    
    rows2 = []
    for (taxid, a, b, rank, name) in rows:
        lineage = make_lineage_from_taxid(taxid)
        rows2.append((taxid, a, b, rank, name, lineage))
        for lintup in lineage:
            if lintup.rank == rank:
                if lintup.name != name:
                    print('DISAGREE: ncbi={}, truthfile={}'.format(lintup.name, name))
    return rows2

truth_lineages = make_truth_lineages('truth_sets/species/Huttenhower_HC1_TRUTH.txt')
print(truth_lineages[:1])

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
[(84980, '1.00000', '0.01000', 'species', 'Desulfotalea psychrophila', (LineagePair(rank='superkingdom', name='Bacteria'), LineagePair(rank='phylum', name='Proteobacteria'), LineagePair(rank='class', name='Deltaproteobacteria'), LineagePair(rank='order', name='Desulfobacterales'), LineagePair(rank='family', name='Desulfobulbaceae'), LineagePair(rank='genus', name='Desulfotalea'), LineagePair(rank='species', name='Desulfotalea psychrophila')))]


## Load LCA gather csv files

In [32]:
def make_lca_gather_lineages(filename):
    rows = load_sourmash_csv(filename)
    rows2 = []
    for d in rows:
        d2 = dict(d)
        
        lineage = []
        for rank in lca_utils.taxlist():
            if rank in d2:
                name = d2.get(rank)
                del d2[rank]
                lineage.append((rank, name))
                
        lineage = [ lca_utils.LineagePair(r, n) for (r, n) in lineage ]
        d2['lineage'] = tuple(lineage)
        rows2.append(d2)
        
    return rows2

make_lca_gather_lineages('output/Huttenhower_HC1.fasta.gz.scaled10k.k51.lca.gather.matches.csv')[:1]

[{'average_abund': '1.0',
  'f_unique_to_query': '0.013083632632119035',
  'f_unique_weighted': '0.013083632632119035',
  'intersect_bp': '510000',
  'lineage': (LineagePair(rank='superkingdom', name='Bacteria'),
   LineagePair(rank='phylum', name='Firmicutes'),
   LineagePair(rank='class', name='Bacilli'),
   LineagePair(rank='order', name='Bacillales'),
   LineagePair(rank='family', name='Bacillaceae'),
   LineagePair(rank='genus', name='Bacillus'),
   LineagePair(rank='species', name='Bacillus subtilis'))}]

In [33]:
ls

Dataset_descriptions.xlsx     get-names-from-gather-csv.py*
README.md                     output/
gather-accessions.txt         output.zip
gather-accessions.txt.taxid   parse.ipynb
gather-lineages.csv           truth_sets/
genbank-lineages.csv


In [34]:
ls output/Huttenhower_HC1*

output/Huttenhower_HC1.fasta.gz.scaled10k.k51.gather.matches.csv
output/Huttenhower_HC1.fasta.gz.scaled10k.k51.lca.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_contigs.scaled10k.k51.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_contigs.scaled10k.k51.lca.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv


In [35]:
ls truth_sets/species/Huttenhower_HC1*


truth_sets/species/Huttenhower_HC1_TRUTH.txt


In [48]:
def compare_gather_to_truth(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    print(len(truth_lineages.intersection(gather_lineages)))
    print(len(truth_lineages.union(gather_lineages)))
    
    print(len(truth_lineages))
    
    print('** in gather but not truth:')
    for diff in gather_lineages - truth_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in truth but not gather:')
    for diff in truth_lineages - gather_lineages:
        print('\t', format_lineage(diff))
    
compare_gather_to_truth('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
93
101
100
** in gather but not truth:
	 Bacteria;Chloroflexi;Chloroflexia;Chloroflexales;Chloroflexaceae;Chloroflexus;Chloroflexus sp. Y-400-fl

** in truth but not gather:
	 Bacteria;Proteobacteria;Alphaproteobacteria;Rickettsiales;Anaplasmataceae;Anaplasma;Anaplasma centrale
	 Bacteria;Chloroflexi;Chloroflexia;Chloroflexales;Chloroflexaceae;Chloroflexus;Chloroflexus aurantiacus
	 Bacteria;Tenericutes;Mollicutes;Mycoplasmatales;Mycoplasmataceae;Mycoplasma;Mycoplasma crocodyli
	 Archaea;Crenarchaeota;Thermoprotei;Thermoproteales;Thermoproteaceae;Pyrobaculum;Pyrobaculum arsenaticum
	 Bacteria;Tenericutes;Mollicutes;Mycoplasmatales;Mycoplasmataceae;Ureaplasma;Ureaplasma urealyticum
	 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanosphaera;Methanosphaera stadtmanae
	 Bacteria;Proteobacteria;Gamm

In [49]:
def compare_lca_gather_to_truth(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_lca_gather_lineages(gather_csv)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    print(len(truth_lineages.intersection(gather_lineages)))
    print(len(truth_lineages.union(gather_lineages)))
    
    print(len(truth_lineages))
    
    print('** in gather but not truth:')
    for diff in gather_lineages - truth_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in truth but not gather:')
    for diff in truth_lineages - gather_lineages:
        print('\t', format_lineage(diff))
    
compare_lca_gather_to_truth('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
98
105
100
** in gather but not truth:
	 Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus;Lactobacillus gallinarum
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus sp. GZT
	 Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio;Vibrio hyugaensis
	 Bacteria;Proteobacteria;Epsilonproteobacteria;Campylobacterales;Helicobacteraceae;Helicobacter;Helicobacter pylori
	 Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Hafniaceae;Edwardsiella;Edwardsiella piscicida

** in truth but not gather:
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus atrophaeus
	 Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Hafniaceae;Edwardsiella;Edwardsiella tarda


In [51]:
def compare_lca_gather_to_gather(lca_gather_csv, gather_csv):
    reg_gather = make_gather_lineages(gather_csv)
    lca_gather = make_lca_gather_lineages(lca_gather_csv)
    
    reg_gather_lineages = set([ row['lineage'] for row in reg_gather ])
    lca_gather_lineages = set([ row['lineage'] for row in lca_gather ])
    
    print(len(lca_gather_lineages.intersection(reg_gather_lineages)))
    print(len(lca_gather_lineages.union(reg_gather_lineages)))
        
    print('** in lca_gather but not reg gather:')
    for diff in lca_gather_lineages - reg_gather_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in gather but not lca gather:')
    for diff in reg_gather_lineages - lca_gather_lineages:
        print('\t', format_lineage(diff))
    
compare_lca_gather_to_gather('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
                            'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')


91
106
** in lca_gather but not reg gather:
	 Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus;Lactobacillus gallinarum
	 Bacteria;Proteobacteria;Alphaproteobacteria;Rickettsiales;Anaplasmataceae;Anaplasma;Anaplasma centrale
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus sp. GZT
	 Bacteria;Chloroflexi;Chloroflexia;Chloroflexales;Chloroflexaceae;Chloroflexus;Chloroflexus aurantiacus
	 Bacteria;Tenericutes;Mollicutes;Mycoplasmatales;Mycoplasmataceae;Mycoplasma;Mycoplasma crocodyli
	 Archaea;Crenarchaeota;Thermoprotei;Thermoproteales;Thermoproteaceae;Pyrobaculum;Pyrobaculum arsenaticum
	 Bacteria;Tenericutes;Mollicutes;Mycoplasmatales;Mycoplasmataceae;Ureaplasma;Ureaplasma urealyticum
	 Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio;Vibrio hyugaensis
	 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanosphaera;Methanosphaera stadtmanae
	 Bacteria;Proteobacteria;Epsilonproteobact

In [53]:
all_truth = """ABRF_MGRG_10ng_TRUTH.txt
ABRF_MGRG_1ng_TRUTH.txt
ABRF_MGRG_5ng_TRUTH.txt
ABRF_MGRG_Half_TRUTH.txt
ABRF_MGRG_Normal_TRUTH.txt
ABRF_MGRG_classIplus_TRUTH.txt
BMI_bmi_reads_TRUTH.txt
BioPool_BioPool_TRUTH.txt
BioPool_BioPool_abundances.txt
Carma_eval_carma_TRUTH.txt
HMP_even_454_SRR072233_TRUTH.txt
HMP_even_illum_SRR172902_TRUTH.txt
Huttenhower_HC-LC_abundances.txt
Huttenhower_HC1_TRUTH.txt
Huttenhower_HC2_TRUTH.txt
Huttenhower_LC1_TRUTH.txt
Huttenhower_LC2_TRUTH.txt
Huttenhower_LC3_TRUTH.txt
Huttenhower_LC4_TRUTH.txt
Huttenhower_LC5_TRUTH.txt
Huttenhower_LC6_TRUTH.txt
Huttenhower_LC7_TRUTH.txt
Huttenhower_LC8_TRUTH.txt
JGI_SRR033547_TRUTH.txt
JGI_SRR033548_TRUTH.txt
JGI_SRR033549_TRUTH.txt
Mavromatis_simHC_TRUTH.txt
Mavromatis_simLC_TRUTH.txt
Mavromatis_simMC_TRUTH.txt
Raiphy_eval_RAIphy_TRUTH.txt
UnAmbiguouslyMapped_ds.7_TRUTH.txt
UnAmbiguouslyMapped_ds.buccal_TRUTH.txt
UnAmbiguouslyMapped_ds.cityparks_TRUTH.txt
UnAmbiguouslyMapped_ds.gut_TRUTH.txt
UnAmbiguouslyMapped_ds.hous1_TRUTH.txt
UnAmbiguouslyMapped_ds.hous2_TRUTH.txt
UnAmbiguouslyMapped_ds.nycsm_TRUTH.txt
UnAmbiguouslyMapped_ds.soil_TRUTH.txt"""

all_truth = all_truth.splitlines()
all_truth = [ x[:-10] for x in all_truth ]
print(all_truth)

['ABRF_MGRG_10ng', 'ABRF_MGRG_1ng', 'ABRF_MGRG_5ng', 'ABRF_MGRG_Half', 'ABRF_MGRG_Normal', 'ABRF_MGRG_classIplus', 'BMI_bmi_reads', 'BioPool_BioPool', 'BioPool_BioPool_abun', 'Carma_eval_carma', 'HMP_even_454_SRR072233', 'HMP_even_illum_SRR172902', 'Huttenhower_HC-LC_abun', 'Huttenhower_HC1', 'Huttenhower_HC2', 'Huttenhower_LC1', 'Huttenhower_LC2', 'Huttenhower_LC3', 'Huttenhower_LC4', 'Huttenhower_LC5', 'Huttenhower_LC6', 'Huttenhower_LC7', 'Huttenhower_LC8', 'JGI_SRR033547', 'JGI_SRR033548', 'JGI_SRR033549', 'Mavromatis_simHC', 'Mavromatis_simLC', 'Mavromatis_simMC', 'Raiphy_eval_RAIphy', 'UnAmbiguouslyMapped_ds.7', 'UnAmbiguouslyMapped_ds.buccal', 'UnAmbiguouslyMapped_ds.cityparks', 'UnAmbiguouslyMapped_ds.gut', 'UnAmbiguouslyMapped_ds.hous1', 'UnAmbiguouslyMapped_ds.hous2', 'UnAmbiguouslyMapped_ds.nycsm', 'UnAmbiguouslyMapped_ds.soil']


In [57]:
triples = []
for prefix in all_truth:
    truth_file = 'truth_sets/species/{}_TRUTH.txt'.format(prefix)
    lca_gather_file = 'output/{}.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv'.format(prefix)
    gather_file = 'output/{}.fasta.gz_reads.scaled10k.k51.gather.matches.csv'.format(prefix)
    
    if os.path.exists(truth_file) and os.path.exists(lca_gather_file) and os.path.exists(gather_file):
        triples.append((truth_file, lca_gather_file, gather_file))
    

In [58]:
for truth_file, lca_gather_file, gather_file in triples:
    # do the computation of precision and recall here
    # Phil TODO

[('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
  'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
  'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv'),
 ('truth_sets/species/Huttenhower_HC2_TRUTH.txt',
  'output/Huttenhower_HC2.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
  'output/Huttenhower_HC2.fasta.gz_reads.scaled10k.k51.gather.matches.csv'),
 ('truth_sets/species/Huttenhower_LC1_TRUTH.txt',
  'output/Huttenhower_LC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
  'output/Huttenhower_LC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv'),
 ('truth_sets/species/Huttenhower_LC2_TRUTH.txt',
  'output/Huttenhower_LC2.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
  'output/Huttenhower_LC2.fasta.gz_reads.scaled10k.k51.gather.matches.csv'),
 ('truth_sets/species/Huttenhower_LC3_TRUTH.txt',
  'output/Huttenhower_LC3.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
  'output/Huttenhower_LC3.fasta.gz_reads.s